<a href="https://colab.research.google.com/github/xiborxu/IST462xibo/blob/main/FinalExam_IST462_Fall2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final exam - IST 462 - Notebook

Submitted by:

Date:

Using the SY36 bus route datasets and the 2019 daily weather conditions for Syracuse data perform the data analysis tasks listed in this document. Perform as many tasks and subtasks as needed to get up to 100 points. If the tasks you complete exceed 100 points, and they are all done correctly, your maximum grade will be capped at 100 points.

Each task has a point value for completing it. There are sub-tasks associated with each task that provide additional points. You can complete a sub-task ONLY if you have completed the main task to which it is associated.

To get all the points for a task or sub-task, in addition to performing the required data analysis mentioned in the task/subtask you must also provide clear documentation of the steps you used in the analysis and in all places possible meaningful graphs that show your results.

**Final exam submission:** Submit your exam work in Notebook format (.ipynb file) ***AND*** in PDF format.

#Import packages

You, of course, can and should add more dependent on what you use in your analysis

In [199]:
%matplotlib inline

import pandas as pd
import numpy as np
import requests
from zipfile import ZipFile
import matplotlib.pyplot as plt
np.set_printoptions(precision=4)
pd.options.display.max_rows = 20

# Loading data sets

In [200]:
# connect colab to google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 2019 Syracuse weather data

In [201]:
import pandas as pd

file = '/content/drive/MyDrive/Final_syracuse_2019_weather.csv'
weather2019 = pd.read_csv(file) #read in the csv file using pandas
weather2019.head()

,STATION,NAME,DATE,AWND,PRCP,SNOW,TAVG,TMAX,TMIN
0,USW00014771,"SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US",1/1/2019,15.66,0.02,0.0,40,53,21
1,USW00014771,"SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US",1/2/2019,5.14,0.00,0.0,24,31,18
2,USW00014771,"SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US",1/3/2019,10.74,0.09,0.3,33,37,30
3,USW00014771,"SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US",1/4/2019,4.70,0.00,0.0,36,49,25
4,USW00014771,"SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US",1/5/2019,5.59,0.00,0.0,33,44,25


In [202]:
weather2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   STATION  365 non-null    object 
 1   NAME     365 non-null    object 
 2   DATE     365 non-null    object 
 3   AWND     365 non-null    float64
 4   PRCP     365 non-null    float64
 5   SNOW     365 non-null    float64
 6   TAVG     365 non-null    int64  
 7   TMAX     365 non-null    int64  
 8   TMIN     365 non-null    int64  
dtypes: float64(3), int64(3), object(3)
memory usage: 25.8+ KB


## Bus ridership data sets

In [ ]:
#Dataframe with bus activity data for 2019 for route SY36
file = '/content/drive/MyDrive/Final_Preprocessed_SY36_2019.csv'
bus_data2019 = pd.read_csv(file) #read in the csv file using pandas
bus_data2019.head()

In [ ]:
bus_data2019.shape

In [ ]:
#Drop some columns that won't be needed
bus_data2019.drop(['SURVEY_DATE','VEHICLE_DESCRIPTION','GARAGE_NAME','DIVISION_NAME','COMMENTS','WHEELCHAIRS','MODIFIED'],axis=1,inplace=True)

In [ ]:
bus_data2019.iloc[:100,]

In [ ]:
bus_data2019.info()

In [ ]:
#Dataframe with bus activity data for 2020 for route SY36
file = '/content/drive/MyDrive//Preprocessed_SY36_2020.csv'
bus_data2020 = pd.read_csv(file) #read in the csv file using pandas
bus_data2020.head()

In [ ]:
#Drop some columns that won't be needed
bus_data2020.drop(['SURVEY_DATE','VEHICLE_DESCRIPTION','GARAGE_NAME','DIVISION_NAME','COMMENTS','WHEELCHAIRS','MODIFIED'],axis=1,inplace=True)

In [ ]:
bus_data2020.head()

In [ ]:
bus_data2020.info()

# Exam tasks

**Task 1 (30 points):** For the year of 2019 determine the number of passengers that board the bus (PASSENGERS_ON) at a particular STOP_ID per day. Use this data to understand how the changes in weather affect the ridership at your selected Bus Stop. Select a bus stop with a daily annual average of at least 5 passengers using it (This means that for any service day of the year, on average, at least 5 passengers boarded the bus from that bus stop).

In [ ]:
# convert the info from TRIP_START_TIME to find the date info
bus_data2019['TRIP_START_TIME'] = pd.to_datetime(bus_data2019['TRIP_START_TIME'])
bus_data2019['DATE'] = bus_data2019['TRIP_START_TIME'].dt.strftime('%m/%d/%Y')
  #create a new column to contain the info
  #I notice I may need to merge the weather data later, so I use same format

In [ ]:
# I notice there are a few columns are actually 2020, I want to delete them
mask = bus_data2019['TRIP_START_TIME'].dt.year != 2019
bus_data2019 = bus_data2019[~mask]

In [ ]:
# Find the stops with daily average Passengers over 5
  # Group by the STOP_ID and DATE info, find out the daily Passengers number
daily_passenger_counts = bus_data2019.groupby(['STOP_ID', 'DATE'])['PASSENGERS_ON'].sum()
  # Calculate the daily average passenger count for each bus stop
average_passengers = daily_passenger_counts.groupby('STOP_ID').mean()
  # Select the stops with average passengers over 5
populated_bus_stops = average_passengers[average_passengers >= 5]
print(len(populated_bus_stops))
print(populated_bus_stops)

In [ ]:
# Now I want to find out the daily passenagers trend
  # I want a subset that only contains the specific STOP info
  # I select 611 here. By info above, its daily average passengers number is 13.93
stop_info = bus_data2019[bus_data2019['STOP_ID'] == 611]
  # Group by the DATE and Passenger number and calculate the sum
daily_passenger_611 = stop_info.groupby('DATE')['PASSENGERS_ON'].sum().reset_index()
daily_passenger_611

***Subtask 1.1 (+10 points):*** Group the activity at the selected bus stop per month and compare against the average temperature for that month.

In [ ]:
#Convert the weather data frame to same format
weather2019['DATE'] = pd.to_datetime(weather2019['DATE'])
weather2019['DATE'] = weather2019['DATE'].dt.strftime('%m/%d/%Y')

In [ ]:
# Merge two dataframes
merged_df = pd.merge(daily_passenger_611,weather2019, on='DATE')
merged_df

In [ ]:
# Find the pattern between passengers number of month
  # Create a 'MONTH' column to use as the index for the pivot table
merged_df['DATE'] = pd.to_datetime(merged_df['DATE'])
merged_df['MONTH'] = merged_df['DATE'].dt.strftime('%m')

# Group by month and calculate the sum of 'PASSENGERS_ON' and the mean of 'Average temperature'
monthly_activity = merged_df.groupby('MONTH').agg({
    'PASSENGERS_ON': 'sum',  # Sum the passengers per month
    'TAVG': 'mean'  # Average temperature for the month
}).reset_index()
monthly_activity

***Subtask 1.2 (+10 points):*** Compare the activity between 2 or more bus stops over each month of the year.

In [ ]:
# I need a Month column in bus_2019 dataframe
bus_data2019['DATE'] = pd.to_datetime(bus_data2019['DATE'])
bus_data2019['MONTH'] = bus_data2019['DATE'].dt.strftime('%m')

# Filter the data for the bus stops 611,612,and 629
stops_compare = bus_data2019[bus_data2019['STOP_ID'].isin([611, 612, 629])]

# Group the data by 'MONTH' and 'STOP_ID' and calculate the sum for 'PASSENGERS_ON', 'PASSENGERS_OFF', and 'PASSENGERS_IN'
activity_comparison = stops_compare.groupby(['MONTH', 'STOP_ID']).agg({
    'PASSENGERS_ON': 'sum',
    'PASSENGERS_OFF': 'sum',
    'PASSENGERS_IN': 'sum'
}).reset_index()

print(activity_comparison)

***Subtask 1.3 (+10 points):*** Determine the 5 bus stops that provide the highest average number of daily passengers during the year.

In [ ]:
# Use the previous series to find out the maximum 5 daily average passengers
stops_df = pd.DataFrame(populated_bus_stops, columns=['PASSENGERS_ON'])

# Find the top 5 stops with the highest number of passengers
top_5_stops = stops_df.nlargest(5, 'PASSENGERS_ON')
print(top_5_stops)



```
# This is formatted as code
```

**Task 2 (30 points):** For the years of 2019 and 2020, determine the number of passengers that board the bus (PASSENGERS_ON) at a particular STOP_ID per week. Select a bus stop where you would expect a high number of users (i.e. Near a shopping mall, school, hospital, etc) and provide proof of how you found this via bus map, etc. Compare the ridership activity between the two years and state a hypothesis as to why changes could have taken place. Provide evidence to support that your hypothesis is likely true. The evidence could be in the form or newspaper articles, government declarations, etc.

In [ ]:
# Generate a DATE info for bus_data2020 with same format
bus_data2020['TRIP_START_TIME'] = pd.to_datetime(bus_data2020['TRIP_START_TIME'])
bus_data2020['DATE'] = bus_data2020['TRIP_START_TIME'].dt.strftime('%m/%d/%Y')
# SO as Month
bus_data2020['DATE'] = pd.to_datetime(bus_data2020['DATE'])
bus_data2020['MONTH'] = bus_data2020['DATE'].dt.strftime('%m')

Bus Station 3761 is at Camillus Commons. (Source: https://www.centro.org/docs/default-source/schedule-documents/auburn-schedules/current-auburn-schedules/aub38.pdf?sfvrsn=5c644e9d_36)
From Google Map, I find Camillus Commons is a business square where there are McDonald, Wendy's Walgreens, Rite Aid, etc., and most importantly, a Walmart there. Also, there is a high school called West Genesee High school is next to it.

In [ ]:
# Create a new column "WEEK_NUMBER" to analysis
bus_data2019['WEEK_NUMBER'] = bus_data2019['DATE'].dt.isocalendar().week
bus_data2020['WEEK_NUMBER'] = bus_data2020['DATE'].dt.isocalendar().week

# Select the info about STOP 3761 to run the analysis
stop_3761_data_2019 = bus_data2019[bus_data2019['STOP_ID'] == 3761]
stop_3761_data_2020 = bus_data2020[bus_data2020['STOP_ID'] == 3761]

#Gouping the data by week number and find the sum of passengers number
stop_2019 = stop_3761_data_2019.groupby('WEEK_NUMBER')['PASSENGERS_ON'].sum()
stop_2020 = stop_3761_data_2020.groupby('WEEK_NUMBER')['PASSENGERS_ON'].sum()
# merge two dataframe together and find out the difference between 2019 and 2020
stop_merged = pd.merge(stop_2019,stop_2020, on = 'WEEK_NUMBER')
stop_merged['Difference'] = stop_merged.apply(lambda x: x['PASSENGERS_ON_y'] - x['PASSENGERS_ON_x'], axis=1)
  # create a lambda function to find out the difference between years
stop_merged.columns = ['2019', '2020', 'Difference']
print(stop_merged)
print(stop_merged['Difference'].nlargest(5))
print(stop_merged['Difference'].nsmallest(5))

The results show that beginning from March, the paseenegers number decreases a lot. This is largely becasue of the COVID that shutdown the social operation. For example, the public school is shutdown including West Genesee High school.I find a source from government webiste by that time: https://www.governor.ny.gov/news/governor-cuomo-signs-executive-order-closing-schools-statewide-two-weeks#:~:text=Governor%20Andrew%20M.,the%20180%2Dday%20instructional%20requirement announcing the shutdown of all public school in New York State.

In [ ]:
bus_data2019.columns

***Subtast 2.1 (+10 points):*** Determine the 10 bus stops that provide the highest average number of weekly passengers for each year. Provide a brief comment on any differences that you find interesting.

In [ ]:
# Group the dataframe by 'STOP_ID' and 'WEEK_NUMBER' and calculate the average number of weekly passengers
stop_average_passengers = bus_data2019.groupby(['STOP_ID', 'WEEK_NUMBER'])['PASSENGERS_ON'].mean().reset_index()
# sort the dataframe by PASSENGER numbers in a descending order
stop_average_passengers = stop_average_passengers.sort_values(by='PASSENGERS_ON', ascending=False)
# Pick top 10 values
top_10_stops = stop_average_passengers.head(10)
print(top_10_stops)

In [ ]:
# Same process for year of 2020
stop_average_passengers = bus_data2020.groupby(['STOP_ID', 'WEEK_NUMBER'])['PASSENGERS_ON'].mean().reset_index()
stop_average_passengers = stop_average_passengers.sort_values(by='PASSENGERS_ON', ascending=False)
top_10_stops = stop_average_passengers.head(10)
print(top_10_stops)

**Task 3 (20 points):** For both the years 2019 and 2020, compute the distance traveled by the buses serving the route per day. Use the data in the SEGMENT_MILES column for this purpose.

In [ ]:
# Grouping the DATE and calculate the sum of segment miles, which will give the daily segent miles number
daily_distance_2019 = bus_data2019.groupby('DATE')['SEGMENT_MILES'].sum().reset_index()
print(daily_distance_2019)

In [ ]:
daily_distance_2020 = bus_data2020.groupby('DATE')['SEGMENT_MILES'].sum().reset_index()
print(daily_distance_2020)

***Subtask 3.1 (+10 points):*** Break down the miles travelled by the buses per month. Assuming an efficiency of 7 miles per gallon and a price of US $3.60 per gallow, compute how much it costs to run the buses per month if they use diesel fuel.

In [ ]:
# Group the data by Month and calcluate the sum of Segment miles, which gives the values of monthly travel distance
monthly_cost_2019 = bus_data2019.groupby('MONTH')['SEGMENT_MILES'].sum().reset_index()
# lambda function to run the maths to finds the price; add the price to the dataframe
monthly_cost_2019['prices'] = monthly_cost_2019.apply(lambda x: f"${x['SEGMENT_MILES'] / 7 * 3.60:.2f}", axis=1)
print(monthly_cost_2019)

monthly_cost_2020 = bus_data2020.groupby('MONTH')['SEGMENT_MILES'].sum().reset_index()
monthly_cost_2020['prices'] = monthly_cost_2020.apply(lambda x: f"${x['SEGMENT_MILES'] / 7 * 3.60:.2f}", axis=1)
print(monthly_cost_2020)

***Subtast 3.2 (+10 points):*** Determine if there are any anomalies (outliers) in the values of the daily distance travelled by the bus each year. Give a possible hypothesis for why they may have occurred and provide evicence to support your hypothesis such as an article or government declaration, etc.

In [ ]:
# to find out the outliers, I draw a time series graph to locate potential outliers.
plt.figure(figsize=(18, 6))
plt.plot(daily_distance_2019['DATE'], daily_distance_2019['SEGMENT_MILES'], marker='o', linestyle='-')
plt.title('Time-Series Plot of Daily Distance Traveled by Bus')
plt.xlabel('Date')
plt.ylabel('Distance Traveled')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
# I did not find any outlier in this graph for 2019

In [ ]:
plt.figure(figsize=(18, 6))
plt.plot(daily_distance_2020['DATE'], daily_distance_2020['SEGMENT_MILES'], marker='o', linestyle='-')
plt.title('Time-Series Plot of Daily Distance Traveled by Bus')
plt.xlabel('Date')
plt.ylabel('Distance Traveled')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In this graph, from 2020 Mid-April to Mid-June, I see a significant decrease at the distance traveled by bus. The time matched the that of COVID so I guess COVID had something to do with it.
I find this website: https://www.syracuse.com/coronavirus/2020/04/centro-bus-routes-reduced-amid-concerns-over-coronavirus.html talking about the centro bus routes reduced during those time because of COVID. This explains the reduction on the bus

**Task 4 (20 points):** For both years 2019 and 2020, study the differences between the time of arrival (TIME_ACTUAL_ARRIVE) and the scheduled bus time (TIME_SCHEDULED) to determine the hours of the day, or the weeks of the year (or some other time period) where significant deviations between the arrival time and the scheduled time appear. Deviations of more than 10 minutes can be considered significant but you can change this limit if it makes sense for your analysis.

In [ ]:
bus_data2019['TIME_ACTUAL_ARRIVE'] = pd.to_datetime(bus_data2019['TIME_ACTUAL_ARRIVE'])
bus_data2019['TIME_SCHEDULED'] = pd.to_datetime(bus_data2019['TIME_SCHEDULED'])
# Calculate the time differences in minutes, and save the values to a new column
bus_data2019['Time_Difference'] = (bus_data2019['TIME_ACTUAL_ARRIVE'] - bus_data2019['TIME_SCHEDULED']).dt.total_seconds() / 60
# Create a new column to identify the hour of the day
bus_data2019['Hour_of_Day'] = bus_data2019['TIME_ACTUAL_ARRIVE'].dt.hour
# Group by the hour of the dat and find out the mean of time difference
hour_time_difference_2019 = bus_data2019.groupby('Hour_of_Day')['Time_Difference'].mean()
hour_time_difference_2019 = pd.DataFrame({'Time Difference': hour_time_difference_2019})
# Set the condition (over 10 mins)
major_difference_hours_2019 = hour_time_difference_2019[(hour_time_difference_2019['Time Difference'] > 10) | (hour_time_difference_2019['Time Difference']< -10)]
major_difference_hours_2019

I believe there should be some mistakes in this graph since the significant amounts of time different appear on some unusual time like 0 and 4 with a huge difference. It probably should be because of the wrong schedule time. The rest of three time indicate the right time. 7 and 15 should be because of the rush hours.

In [ ]:
bus_data2020['TIME_ACTUAL_ARRIVE'] = pd.to_datetime(bus_data2020['TIME_ACTUAL_ARRIVE'])
bus_data2020['TIME_SCHEDULED'] = pd.to_datetime(bus_data2020['TIME_SCHEDULED'])

# Calculate the time differences in minutes
bus_data2020['Time_Difference'] = (bus_data2020['TIME_ACTUAL_ARRIVE'] - bus_data2020['TIME_SCHEDULED']).dt.total_seconds() / 60
bus_data2020['Hour_of_Day'] = bus_data2020['TIME_ACTUAL_ARRIVE'].dt.hour
hour_time_difference_2020 = bus_data2020.groupby('Hour_of_Day')['Time_Difference'].mean()
hour_time_difference_2020 = pd.DataFrame({'Time Difference': hour_time_difference_2020})
major_difference_hours_2020 = hour_time_difference_2020[(hour_time_difference_2020['Time Difference'] > 10) | (hour_time_difference_2020['Time Difference']< -10)]
major_difference_hours_2020

I believe the same thing, wrong schedule time here to make this happen.

***Subtask 4.1 (+10 points):*** For 2019 only investigate if there is any relationship between weather conditions (termperature, wind, snow) with the deviations between time of arrival and scheduled bust time. Justify your analysis with graphs and documentation.

In [ ]:
# Merge the weather and bus info data together
weather2019['DATE'] = pd.to_datetime(weather2019['DATE'])
merged_2019 = pd.merge(weather2019, bus_data2019, on = 'DATE')
# group by the DATE and find out the mean of AWND, PRCP,SNOW and TAVG.
weather_difference = merged_2019.groupby('DATE')[['AWND','PRCP','SNOW','TAVG','Time_Difference']].mean().reset_index()
weather_difference

In [ ]:
corr = weather_difference.corr()
corr.style.background_gradient(cmap = 'coolwarm')

From the correlation graph, the time difference has a higher relationship with average wind speed and average temperature.

***Subtask 4.2 (+10 points):*** Investigate if the VEHICLE_NUMBER or OPERATOR_ID have an influence in the deviations between time of arrival and scheduled bus time. Justify your analysis with graphs and documentation.

In [ ]:
#merged_2019.head()
# same process as above, group by the Vehicle number and find the mean of time difference
veh_num_difference = merged_2019.groupby('VEHICLE_NUMBER')['Time_Difference'].mean()

# the output is a series but I need a dataframe for graph. Convert it to a df
veh_num_difference = veh_num_difference.reset_index()
veh_num_difference.columns = ['Vehicle Number', 'Time Difference']
# draw the relationship graph
corr = veh_num_difference.corr()
corr.style.background_gradient(cmap = 'coolwarm')

From the correlation graph, the relationship between vehcile number and time difference is only -0.45,which is not a high correlation.

In [ ]:
# same process for the operation ID
oper_id_difference = merged_2019.groupby('OPERATOR_ID')['Time_Difference'].mean()
oper_id_difference = oper_id_difference.reset_index()
oper_id_difference.columns = ['Operation ID', 'Time Difference']

corr = oper_id_difference.corr()
corr.style.background_gradient(cmap = 'coolwarm')

The correlation is 0.056, which has a strong correlation between the time difference and operation ID.